# Generating Hard to Approximate Functions using Dynamical Systems

### Background 

Here we generate data points from hard to approximate function for any given architecture of a neural network. We assume the neural network to have width $u$ per layer and $l$ layers. It was shown in our work titled "Better depth-width trade-offs for neural networks through the lens of dynamical systems" at ICML 2020, that we can construct simple, bounded Lipschitz functions that have an odd period p (we say that a function has a period $p$ an integer greater than or equal to 1, when there is some $x_0$ such that $f^p(x_0)=x_0$ and $\{f(x_0),f^2(x_0),\ldots,f^{p-1}(x_0)\}$ are distinct and $f^k(.)$ represents the $k^{th}$ composition of the function $f$). This has non-vanishing oscillations that grow at the rate of $\rho$ such that $\sqrt{2} \leq \rho \leq \phi$, where $\phi = \frac{1+\sqrt{5}}{2}$ (golden ratio). We show in our paper that when the Lipschitzness of the function $L$ is equal to $\rho$ (the growth rate of oscillations) for functions with odd periodic points p and when the neural network satisfies the inequality $(2u)^l\leq \frac{\rho^t}{\alpha}$, with $\alpha > 2$, then the regression error between the function $f^t(.)$ and any function induced by the neural network of the particular architecture will be a non-zero constant that does not depend on $t$ (the number of compositions of the function $f$). This intuitively means that given any architecture (a specification of $u$ and $l$), we can find a $t$, the number of compositions required to make sure that the above condition is satisfied. We can set $\alpha = 8$ (arbitrarily).

### Method to Generate Data

The functions that we propose that satisfy all the required qualities described above. The function $f: [-1,1] \rightarrow [-1,1]$ is defined to be $f(x)= \rho|x|-1$. Here the Lipschitz constant of this function is $\rho$ and the growth rate of its oscillations is $\rho$. This function has period 3, when $\rho = 1.618$ (the golden ratio), the highest growth rate.
To find $t$, we simply do $t \geq \frac{l\log(2u) + \log(8)}{\log(\rho)}$. When $u$ and $l$ are large, $t$ is large. But how do we take large compositions? Python symbolic packages are not scalable to do the job. The easy work around is to actually treat it as a dynamical system. In the sense we randomly initialize $x_0$ in the interval $[-1,1]$ and then we implement the recursion $x_{t} = f(x_{t-1})= \rho|x_{t-1}|-1$. We take sufficiently large number of $x_0's$ and repeat this to generate our data set that consists of pairs $\{x^{(i)},f^t(x^{(i)})\}$, where $\{x^{(1)},x^{(2)}\ldots,x^{(n)}\}$ are the $n$ different starting points randomly sampled in $[-1,1]$.

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### Data Generation Code

In [39]:
#Input data goes here-Generate data for the regression task
np.random.seed(42);
n=10000 #number of starting points (essentially size of your data set).
rho=1.618 #Corresponds to period 3. 
t=40; #No of compositions of f, set according to your NN architecture.
y_data=np.zeros((n,1)) #The values f^t(x)
x_data=-1+2*np.random.uniform(size=(n,1)); #The starting data points.
for i in range(n):
 x_temp=x_data[i,0]
 for j in range(t): 
    x_temp=-(1-rho*np.abs(x_temp)); #f(x)=rho|x|-1
 y_data[i,0]=x_temp;
#Use this training data on your favourite NN architecture and see how the training and generalization capabilities are.

### Obtain growth rates for other periods

In [38]:
p = 3 #Periodic point required for the function $f$. Lower the value, higher the growth rate! Always set p to be an odd integer > 1.
func = lambda tau : tau**p - 2*tau**(p-2) - 1 
init_guess = 1.7 #This is set to a high value > 1.6 to allow to find solutions that are between sqrt(2) and 1.618. The largest root is always > 1.
tau_solution = fsolve(func, init_guess)
rho=np.abs(tau_solution)
#Before feeding into the data generation above, check if rho lies between sqrt(2) and 1.618.